<a href="https://colab.research.google.com/github/yyaaoonngg/DL/blob/main/3_data_matching(dr14).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing packages
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from astropy.coordinates import search_around_sky, SkyCoord
from astropy import units as u
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 999)

In [ ]:
dr14_BH = pd.read_csv('/content/drive/MyDrive/DL/Data/dr14_BH.csv')
dr14_LC = pd.read_csv('/content/drive/MyDrive/DL/Data/clean_LC.csv')

In [ ]:
dr14_LC

,objectId,ra,dec,u_band,g_band,r_band,i_band,z_band,ug,gr,ri,iz,zu
0,271386,359.99735,1.807657,23.69984,21.964514,20.421870,18.904140,18.213037,1.735326,1.542644,1.517730,0.691103,-5.486803
1,271388,359.99169,0.969968,20.27555,17.727280,16.289740,15.616840,15.253790,2.548270,1.437540,0.672900,0.363050,-5.021760
2,271389,359.98092,0.652865,19.59781,19.171840,19.258650,19.395890,19.594210,0.425970,-0.086810,-0.137240,-0.198320,-0.003600
3,271390,359.97467,-1.066557,19.40718,18.168272,17.653803,17.498888,17.421219,1.238908,0.514469,0.154915,0.077669,-1.985961
4,271391,359.97300,0.203557,19.43786,18.553917,18.252184,18.175772,18.147707,0.883943,0.301733,0.076412,0.028065,-1.290153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
442689,1468017,309.23870,0.028991,20.68143,20.310740,20.156620,19.788150,19.487930,0.370690,0.154120,0.368470,0.300220,-1.193500
442690,1468018,309.22960,0.171063,19.40578,19.420370,19.313830,19.416860,19.460500,-0.014590,0.106540,-0.103030,-0.043640,0.054720
442691,1468019,309.22200,-0.160658,20.18258,20.105960,19.982310,19.651100,19.664880,0.076620,0.123650,0.331210,-0.013780,-0.517700
442692,1468020,309.21940,-0.479245,20.70151,20.390630,20.127720,20.142670,19.929070,0.310880,0.262910,-0.014950,0.213600,-0.772440


In [ ]:
# astropy의 SkyCoord를 이용하여, 2개의 데이터셋의 데이터 속성을 matching
COORD1 = SkyCoord(dr14_BH['ra'], dr14_BH['dec'], frame = 'icrs', unit = 'deg')
COORD2 = SkyCoord(dr14_LC['ra'], dr14_LC['dec'], frame = 'icrs', unit = 'deg')
IDX1, IDX2, OTHER1, OTHER2 = search_around_sky(COORD1, COORD2, seplimit= 0.5 * u.arcsec)

In [ ]:
# 매치된 것을 위한 열 생성
X_TRAIN = []
for i in range(len(IDX1)):
  result = dr14_BH.iloc[IDX1[i]].append(dr14_LC.iloc[IDX2[i]])
  X_TRAIN.append(result)

X_TRAIN = pd.concat(X_TRAIN, axis = 1) # 동일한 형태의 dataframe 합치는 모듈
X_TRAIN =  X_TRAIN.T # 데이터 프레임 전치 (행 <-> 열)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-8-4d9334037061>:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = dr14_BH.iloc[IDX1[i]].append(dr14_LC.iloc[IDX2[i]])
<ipython-input-8-4d9334037061>:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = dr14_BH.iloc[IDX1[i]].append(dr14_LC.iloc[IDX2[i]])
<ipython-input-8-4d9334037061>:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = dr14_BH.iloc[IDX1[i]].append(dr14_LC.iloc[IDX2[i]])
<ipython-input-8-4d9334037061>:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = dr14_BH.iloc[IDX1[i]].append(dr14_LC.iloc[IDX2[i]])
<ipython-input-8-4d9334037061>:4: Future

In [ ]:
# pd df에서 loc -> indexing
X_TRAIN = X_TRAIN.loc[:, ~X_TRAIN.columns.str.contains('^Unnamed')]

In [ ]:
# 우리는 RA/DEC을 비교할 수 있다. 데이터가 맞게 matching됐다는 것을 확인시켜주는
X_TRAIN = X_TRAIN.drop(columns= ['SDSS_ID','ID'])

In [ ]:
X_TRAIN.head()
# train_id와 objectId 차이
# 데이터도 다른 듯함.....

,ra,dec,Mass,z,ERR,M_i,objectId,ra,dec,u_band,g_band,r_band,i_band,z_band,ug,gr,ri,iz,zu
0,0.005754,-1.325013,8.744595,2.328,0.059799,-26.184576,1444816.0,0.005746,-1.325009,20.64888,19.894005,19.910748,19.799997,19.557253,0.754875,-0.016743,0.110751,0.242744,-1.091627
1,0.006782,0.583213,8.3602,2.271102,0.055385,-24.261814,1444814.0,0.006855,0.583184,23.53959,21.77645,21.7755,21.60524,21.337786,1.76314,0.00095,0.17026,0.267454,-2.201804
2,0.005317,-2.033272,8.657381,1.36,0.064861,-25.890327,1444817.0,0.005317,-2.033273,20.55112,20.01035,19.257164,19.013952,18.924904,0.54077,0.753186,0.243212,0.089048,-1.626216
3,0.009139,-2.329783,8.498153,1.966623,0.040606,-24.97323,1444812.0,0.009139,-2.329783,21.09795,20.90291,20.81347,20.537844,20.45843,0.19504,0.08944,0.275626,0.079414,-0.63952
4,0.013238,1.252965,8.985262,2.354,0.163448,-25.45225,1444810.0,0.013229,1.252967,21.76721,21.328455,21.193518,21.227804,20.877909,0.438755,0.134937,-0.034286,0.349895,-0.889301


In [ ]:
# head에서 볼 수 있는 ra-dec 같은 두 자료를 통해 데이터가 일치하는지 확인

'''
data = [Ra, Dec, z, ERR, M_i,
        ID, Ra, Dec,
        u, g, r, i, z
        ug, gr,ri, iz, zu]
'''

our = [0.006782,	0.583213,	8.3602,	2.271102,	0.055385,	-24.261814,
      1444814.0,	0.006855,	0.583184,
      23.53959,	21.77645,	21.7755,	21.60524,	21.337786,
      1.76314,	0.00095,	0.17026,	0.267454,	-2.201804]

cpr = [0.0067823,	0.583213,	8.3602,	2.2711,	0.0553846,	-24.2618,
      23207,	0.00685539,	0.583184,
      23.4436,	22.2629,	22.2395,	22.1408,	21.5903,
      1.18065, 0.02345,	0.09865,	0.55052,	-1.85327]

# 색지수 계산법 같으나 ugriz 값이 미세하게 달라서 색지수 값도 달라짐, 이외 물리량들은 거의 같으나 관측이 정확해짐에 따라 약간 다르다
# 코드를 따라할 수는 있겠지만 확실히 잘 따라하고 있는지 확인하기는 약간 어려울 듯

In [ ]:
X_TRAIN = X_TRAIN.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
print(X_TRAIN.shape) # 깃허브에서의 X_TRAIN.shape = (20549, 19) ;https://github.com/snehjp2/AGNet/blob/main/src/notebooks/match_dr14_LC_BH.ipynb

(49581, 19)


In [ ]:
# 데이터를 훈련 데이터와 테스트 데이터로 나눔
train, test = train_test_split(X_TRAIN, test_size = 0.15)

# check
test.shape[0] + train.shape[0] == X_TRAIN.shape[0]

True

In [ ]:
X_TRAIN.to_csv('/content/drive/MyDrive/DL/Data/matched_dr14.csv')

In [ ]:
train = train.dropna()
test = test.dropna()
train.to_csv('/content/drive/MyDrive/DL/Data/TRAIN_dr14.csv')
test.to_csv('/content/drive/MyDrive/DL/Data/TEST_dr14.csv')